In [1]:
import os,sys; sys.path.insert(0,os.path.abspath('../..'))
import os,sys; sys.path.insert(0,os.path.abspath('/Users/ryan/github/yapmap'))

from borges import *
from chadwyck import *
from yapmap import *

In [2]:
class ChadwyckCorpus(BaseCorpus):
    id='chadwyck'
    name='Chadwyck'
    ext_raw = '.new'

    def compile_metadata(self, num_proc=4, lim=None, **kwargs):
        res=pmap(compile_metadata, self.filenames_raw[:lim], num_proc=num_proc, **kwargs)
        df=pd.DataFrame(res).set_index(KEY_ID)
        df.to_csv(self.path_metadata)

    def compile_pages(self, num_proc=4, lim=None, **kwargs):
        pmap(
            compile_pages, 
            self.filenames_raw[:lim], 
            num_proc=num_proc, 
            **kwargs
        )

In [3]:
C=ChadwyckCorpus()

In [4]:
# C.compile_metadata(lim=None, num_proc=8)
C.compile_pages(num_proc=8,lim=None)

 37%|███▋      | 132/359 [03:00<11:35,  3.06s/it] 

In [ ]:
# %%timeit
# solr=get_meta_solr()
# res=solr.search('id:_chadwyck/Eighteenth-Century_Fiction/haywood.11')
# for x in res: break
# x

In [ ]:
get_first(get_page_solr().search('text_id:_chadwyck/Eighteenth-Century_Fiction/haywood.11'))

{'id': '_chadwyck/Eighteenth-Century_Fiction/haywood.11/page/9',
 'text_id': ['_chadwyck/Eighteenth-Century_Fiction/haywood.11'],
 'page_num': ['9'],
 'page': ['<pb n="9">\n<ps>it in such a manner, that no Mortal was\nable to resist their Force—And, indeed,\nwhen Passion enters the Soul by such gentle and\nunperceiv\'d Degrees, it generally takes a surer hold,\nand is with much less ease extirpated, than when\nit rushes all at once upon us, and boldly tells us\nthat we must obey. </ps></s></p>\n<p><s><hi r="smcap">Lasselia</hi> being such, and infinitely more agreeable\nthan I have power to represent her, \'tis\neasy to believe she was not without a very great\nNumber of Adorers: Both her Parents being dead,\nMadam <hi r="italic">de</hi> <hi r="smcap">Montespan</hi> was never at rest for\nthe Sollicitations of those whom the Perfections of\nher beautiful Neice had attracted to desire her in\nMarriage: but that Lady having Affairs enough\nof her own to manage, troubled not herself much\